In [23]:
import os
import pandas as pd
import glob
from tqdm import tqdm  # Progress bar

def clean_parquet_files(input_dir="data", output_dir="cleaned_data"):
    """
    This function processes multiple Parquet files from the 'data' folder by applying data cleaning steps.
    Cleaned files are saved to the 'cleaned_data' directory.
    """

    # -------------------------
    # 1. Ensure output directory exists
    # -------------------------
    os.makedirs(output_dir, exist_ok=True)

    # -------------------------
    # 2. Find all Parquet files in the input directory
    # -------------------------
    file_list = glob.glob(os.path.join(input_dir, "yellow_tripdata_2024-*.parquet"))

    if not file_list:
        print("❌ No files found in './data'. Please check the folder and file names!")
        return

    print(f"🔍 Found {len(file_list)} files in '{input_dir}/'. Starting data cleaning...\n")

    # -------------------------
    # 3. Process all files with a single progress bar
    # -------------------------
    with tqdm(total=len(file_list), desc="Cleaning Progress", unit="file") as pbar:

        for file_path in file_list:

            # -------------------------
            # 3.1 Read Parquet file
            # -------------------------
            df = pd.read_parquet(file_path, engine="pyarrow")  # Use "fastparquet" if needed

            # -------------------------
            # 3.2 Handle missing values
            # -------------------------
            df["passenger_count"] = df["passenger_count"].fillna(1) # Set NaN to default 1
            df["trip_distance"] = df["trip_distance"].fillna(0)
            df.fillna(0, inplace=True)

            # -------------------------
            # 3.3 Remove unnecessary columns
            # -------------------------
            if "store_and_fwd_flag" in df.columns:
                df.drop(columns=["store_and_fwd_flag"], inplace=True)

            # -------------------------
            # 3.4 Convert datetime columns
            # -------------------------
            df["pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
            df["dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

            # -------------------------
            # 3.5 Calculate trip duration in minutes
            # -------------------------
            df["trip_duration"] = (df["dropoff_datetime"] - df["pickup_datetime"]).dt.total_seconds() / 60

            # -------------------------
            # 3.6 Remove extreme values
            # -------------------------
            df = df[df["trip_distance"] >= 0.05] # Remove trips with distance < 0.05 miles
            df = df[df["fare_amount"] > 0]  # Remove negative fare amounts
            df = df[(df["passenger_count"] > 0)]  # Keep passenger count bigger than 0
            df = df[(df["trip_duration"] >= 1) & (df["trip_duration"] <= 180)]  # Keep trip duration (1-180 mins)

            # -------------------------
            # 3.7 Drop duplicate rows
            # -------------------------
            df.drop_duplicates(inplace=True)

            # -------------------------
            # 3.8 Rearrange columns
            # -------------------------
            df = df[[
                "VendorID", "pickup_datetime", "dropoff_datetime", "trip_duration",
                "passenger_count", "trip_distance", "RatecodeID", "PULocationID",
                "DOLocationID", "payment_type", "fare_amount", "total_amount",
                "tip_amount", "tolls_amount", "congestion_surcharge"
            ]]

            # -------------------------
            # 3.9 Save cleaned file in the output directory
            # -------------------------
            output_filename = "cleaned_" + os.path.basename(file_path)
            output_path = os.path.join(output_dir, os.path.basename(output_filename))
            df.to_parquet(output_path, index=False)

            # Update progress bar
            pbar.update(1)

    print("\n🎉 All files have been successfully cleaned and saved in 'cleaned_data/'!")

# -------------------------
# 4. Run the function
# -------------------------
clean_parquet_files()


🔍 Found 12 files in 'data/'. Starting data cleaning...



Cleaning Progress: 100%|██████████| 12/12 [01:27<00:00,  7.30s/file]


🎉 All files have been successfully cleaned and saved in 'cleaned_data/'!
